In [13]:
import cv2
import cv2.aruco as aruco
import re
import numpy as np

In [14]:
def get_aruco_dict_from_filename(filename):
    match = re.search(r'(\d+)x(\d+)_(\d+)', filename)
    if not match:
        raise ValueError("Invalid filename format")

    bits = int(match.group(1))
    total = int(match.group(3))

    key = f'DICT_{bits}X{bits}_{total}'
    return getattr(aruco, key)

In [15]:
def quad_size(quad):
    quad = quad.reshape(4,2)
    return min(
        np.linalg.norm(quad[0]-quad[1]),
        np.linalg.norm(quad[1]-quad[2])
    )

def enhance_for_large_markers(gray):
    clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
    eq = clahe.apply(gray)

    blur = cv2.GaussianBlur(eq, (5,5), 0)
    sharp = cv2.addWeighted(eq, 1.15, blur, -0.15, 0)

    return sharp


In [ ]:



filename = "ArUco_videos/Video7(5x5_250).mp4"
cap = cv2.VideoCapture(filename)
get_aruco_dict_from_filename(filename)

dict_id = get_aruco_dict_from_filename(filename)
dictionary = aruco.getPredefinedDictionary(dict_id)
parameters = aruco.DetectorParameters()
detector = aruco.ArucoDetector(dictionary, parameters)

print("Video opened:", cap.isOpened())

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame2 = frame.copy()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# 1. Mild contrast normalization
    clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
    gray_eq = clahe.apply(gray)

    # 2. Edge-preserving denoising (very mild)
    denoised = cv2.bilateralFilter(gray_eq, d=7, sigmaColor=50, sigmaSpace=50)

    # 3. Mild unsharp masking (geometry-preserving)
    blur = cv2.GaussianBlur(denoised, (5,5), 0)
    sharp = cv2.addWeighted(denoised, 1.2, blur, -0.2, 0)
    

    corners, ids, rejected = detector.detectMarkers(gray)
    small_candidates = any(quad_size(q) < 25 for q in rejected)
    if ids is None and not small_candidates:
        processed = enhance_for_large_markers(gray)
        corners, ids, rejected = detector.detectMarkers(processed)
        aruco.drawDetectedMarkers(frame, corners, ids)
    else:
        
        corners, ids, rejected = detector.detectMarkers(gray)
        aruco.drawDetectedMarkers(frame, corners, ids)


    cv2.imshow("ArUco Detection", frame)


    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Video opened: True
